In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os, sys

# Ajouter le dossier racine au path pour accéder à src/
sys.path.append(os.path.abspath(".."))

from src.models.eegnet import EEGNetLight

# Charger les données EEG déjà prétraitées
data = np.load("../../data/processed/subject_1_epochs.npz")
X = data["X"]  # (N, C, T)
y = data["y"]  # (N,)

# 🔧 Corriger les labels si nécessaire (ex: 1, 2, 3 → 0, 1, 2)
y = y - y.min()

# Préparation des tenseurs
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # [N, 1, C, T]
y_tensor = torch.tensor(y, dtype=torch.long)

# Dataset & DataLoader
ds = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(ds, batch_size=8, shuffle=True)

# Initialiser le modèle
n_channels, n_timepoints = X.shape[1], X.shape[2]
n_classes = len(np.unique(y))
model = EEGNetLight(n_channels=n_channels, n_timepoints=n_timepoints, n_classes=n_classes)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# Boucle d'entraînement
for epoch in range(10):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for xb, yb in loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (preds.argmax(1) == yb).sum().item()
        total += yb.size(0)

    acc = 100 * correct / total
    print(f"🧠 Epoch {epoch+1:02d} — Loss: {total_loss:.4f} — Acc: {acc:.2f}%")
